In [6]:
import pandas as pd
from mysql import connector

df = pd.read_csv("ap_bus_details.csv")
df = pd.read_csv("telengana_bus_details.csv")
df = pd.read_csv("kerala_bus_details.csv")
df = pd.read_csv("south_bengal_bus_details.csv")
df = pd.read_csv("bihar_bus_details.csv")
df = pd.read_csv("himachal_pradesh_bus_details.csv")
df = pd.read_csv("rajasthan_bus_details.csv")
df = pd.read_csv("punjab_bus_details.csv")
df = pd.read_csv("assam_bus_details.csv")
df = pd.read_csv("goa_bus_details.csv")

csv_files = ["ap_bus_details.csv", "telengana_bus_details.csv", "kerala_bus_details.csv", "south_bengal_bus_details.csv", "bihar_bus_details.csv", "himachal_pradesh_bus_details.csv", "rajasthan_bus_details.csv" , "punjab_bus_details.csv", "assam_bus_details.csv" , "goa_bus_details.csv"]
df = [pd.read_csv(file) for file in csv_files]

combined_df = pd.concat(df, ignore_index=True)
combined_df.to_csv("bus_routes.csv", index=False)

id_column = pd.Series(range(1, len(df) + 1), name='id')
df = pd.concat([id_column, df], axis=1)
df = pd.read_csv("bus_routes.csv")

df['Price'] = df['Price'].str.replace('INR ', '')
df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')

myconnection=connector.connect(
    host='localhost',
    user='root',
    password='123456789',
    database='redbus'
)

df = pd.read_csv("bus_routes.csv")
df = df.dropna() #drop null values
a = ",".join(f"{i} {j}"
for i,j in zip(df.columns,df.dtypes)).replace("float64","float").replace("object","text").replace("int64","int")
table_name = "bus_routes"
myconnection.cursor().execute(f"create table {table_name} ({a})")
for i in range(len(df)):
    myconnection.cursor().execute(f"insert into {table_name} values {tuple(df.iloc[i])}")
    myconnection.commit()

<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
C:\Users\manoj\AppData\Local\Temp\ipykernel_3624\3864390592.py:26: SyntaxWarning: invalid escape sequence '\d'
  df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')
C:\Users\manoj\AppData\Local\Temp\ipykernel_3624\3864390592.py:26: SyntaxWarning: invalid escape sequence '\d'
  df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [4]:
import pandas as pd
from mysql import connector

csv_files = [
    "ap_bus_details.csv", "telengana_bus_details.csv", "kerala_bus_details.csv",
    "south_bengal_bus_details.csv", "bihar_bus_details.csv",
    "himachal_pradesh_bus_details.csv", "rajasthan_bus_details.csv",
    "punjab_bus_details.csv", "assam_bus_details.csv", "goa_bus_details.csv"
]

dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        df['Price'] = df['Price'].str.replace('INR ', '', regex=False)
        df['Seat_Availability'] = df['Seat_Availability'].str.extract(r'(\d+)', expand=False)
        dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.drop_duplicates(inplace=True)
combined_df.dropna(inplace=True) 


combined_df.rename(columns={
    'Route_Name': 'route_name',
    'Route_Link': 'route_link',
    'Bus_Name': 'busname',
    'Bus_Type': 'bustype',
    'Departing_Time': 'departing_time',
    'Duration': 'duration',
    'Reaching_Time': 'reaching_time',
    'Star_Rating': 'star_rating',
    'Price': 'price',
    'Seat_Availability': 'seat_availability'
}, inplace=True)

combined_df['price'] = pd.to_numeric(combined_df['price'], errors='coerce')
combined_df['seat_availability'] = pd.to_numeric(combined_df['seat_availability'], errors='coerce').fillna(0).astype(int)

myconnection = connector.connect(
        host='localhost',
        user='root',
        password='123456789',
        database='redbus'
    )
cursor = myconnection.cursor()

create_table_query = """
    CREATE TABLE IF NOT EXISTS bus_routes (
        id INT AUTO_INCREMENT PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        busname TEXT,
        bustype TEXT,
        departing_time TIME,
        duration TEXT,
        reaching_time TIME,
        star_rating FLOAT,
        price DECIMAL(10, 2),
        seat_availability INT
    );
    """
cursor.execute(create_table_query)
print("Table `bus_routes` created successfully!")

try:
    data = [tuple(row) for row in combined_df.to_numpy()]
    placeholders = ", ".join(["%s"] * len(combined_df.columns))
    insert_query = f"INSERT INTO bus_routes ({', '.join(combined_df.columns)}) VALUES ({placeholders})"

    cursor.executemany(insert_query, data)
    myconnection.commit()

except connector.Error as e:
    print(f"Error inserting data: {e}")
    
cursor.close()
myconnection.close()

print('Done!')


Table `bus_routes` created successfully!
Done!
